In [53]:
thisyear = 2012
do_these_states = ['06', '11', '24', '26', '29', '34', '39', '42', '51'
    #'01', '02', '04', '05']
#    '24'#,
    #'01', '02', '04', '05', #, 
    #'06' #,
     #'08', '09', '10', '11', '12' #, 
     #'13', '15', '16', '17', '18' #, 
     #'19', '20', '21', '22', '23' #, 
#     '25', '26', '27', '28' #, 
#      '29', '30', '31', '32', '33' #, 
#      '34', '35', '36', '37', '38' #, 
#      '39', '40', '41', '42', '44' #, 
#      '45', '46', '47' #, 
#.     '48' #, 
#      '49', '50', '51', '53' #, 
#      '54', '55', '56', '60', '66', '69', '72', '78'
 ]


import os
import shutil
import zipfile
import pandas 
import geopandas
#import gzip
import numpy as np
import urllib3
import certifi
from bs4 import BeautifulSoup as bs
import time

script_dir = '/home/idies/workspace/21cc/raddick/'
data_dir = '/home/idies/workspace/21cc/Data/Census/'
shapefiledir = data_dir + 'Shapefiles/TIGER/'
#extras_dir = '/home/idies/workspace/Storage/raddick/census/extras/'

if not (os.path.exists(shapefiledir)):
    os.makedirs(shapefiledir)
os.chdir(shapefiledir)

print('setting up HTTPS pool for downloading...\n')
c = urllib3.PoolManager(cert_reqs="CERT_REQUIRED", ca_certs=certifi.where())

print('In directory: '+os.getcwd())

setting up HTTPS pool for downloading...

In directory: /home/idies/workspace/21cc/Data/Census/Shapefiles/TIGER


# Create directory structure if needed

In [54]:
yeardir = shapefiledir + '{0:.0f}/'.format(thisyear)
tractdir = yeardir + 'TRACT/'
bgdir = yeardir + 'BG/'
placedir = yeardir + 'PLACE/'
#coastdir = yeardir + 'COASTLINE/'
if (thisyear >= 2020):
    uacdir = yeardir + 'UAC/'
else:
    uacdir = yeardir + 'CBSA/'
areawaterdir = yeardir + 'AREAWATER/'
#linearwaterdir = yeardir + 'LINEARWATER/'
roaddir = yeardir + 'ROADS/'
#pumadir = yeardir + 'PUMA/'
cddir = yeardir + 'CD/'
zcta5dir = yeardir + 'ZCTA5/'

if not(os.path.exists(tractdir)):
    os.makedirs(tractdir)  
# if not(os.path.exists(bgdir)):
#     os.makedirs(bgdir)
if not(os.path.exists(placedir)):
    os.makedirs(placedir)
# if not(os.path.exists(coastdir)):
#     os.makedirs(coastdir)
# if not(os.path.exists(uacdir)):
#     os.makedirs(uacdir)
if not(os.path.exists(areawaterdir)):
    os.makedirs(areawaterdir)
# if not(os.path.exists(linearwaterdir)):
#     os.makedirs(linearwaterdir)
if not(os.path.exists(roaddir)):
    os.makedirs(roaddir)
# if not(os.path.exists(pumadir)):
#     os.makedirs(pumadir)
# if not(os.path.exists(cddir)):
#     os.makedirs(cddir)
# if not(os.path.exists(zcta5dir)):
#     os.makedirs(zcta5dir)

g = 0

os.chdir(yeardir)
os.getcwd()

'/home/idies/workspace/21cc/Data/Census/Shapefiles/TIGER/2012'

# What shapefile levels are available?

In [ ]:
# listurl = 'https://www2.census.gov/geo/tiger/TIGER{0:.0f}/'.format(thisyear)

# resp = c.request("GET", listurl)
# soup = bs(resp.data, 'html.parser')
# thetable = soup.find('table')
# for i in range(0, len(thetable)):
#     if (len(thetable.contents[i]) > 1):
#         tdlist = thetable.contents[i].findAll('td')
#         if (len(tdlist) > 1):
#             print(tdlist[1].get_text())

# Get state and county shapefiles

### Get from www2.census.gov and unzip

In [55]:
s = time.time()
print('getting state shapefiles...')
theurl = 'https://www2.census.gov/geo/tiger/TIGER{0:.0f}/STATE/tl_{0:.0f}_us_state.zip'.format(thisyear)
outfile = '{0:}tl_{1:.0f}_us_state.zip'.format(yeardir, thisyear)

r = c.request('GET', theurl, preload_content=False)
with open(outfile, 'wb') as out:
    while True:
        data = r.read()
        if not data:
            break
        out.write(data)

r.release_conn()


print('getting county shapefiles...')
theurl = 'https://www2.census.gov/geo/tiger/TIGER{0:.0f}/COUNTY/tl_{0:.0f}_us_county.zip'.format(thisyear)
outfile = '{0:}tl_{1:.0f}_us_county.zip'.format(yeardir, thisyear)

r = c.request('GET', theurl, preload_content=False)

with open(outfile, 'wb') as out:
    while True:
        data = r.read()
        if not data:
            break
        out.write(data)

r.release_conn()

os.chdir(yeardir)

print('Extracting state shapes zipfile...')
thezipfile = zipfile.ZipFile(yeardir+'tl_{0:.0f}_us_state.zip'.format(thisyear))
thezipfile.extractall()
thezipfile.close()

print('Extracting county shapes zipfile...')
thezipfile = zipfile.ZipFile(yeardir+'tl_{0:.0f}_us_county.zip'.format(thisyear))
thezipfile.extractall()
thezipfile.close()

os.remove(yeardir+'tl_{0:.0f}_us_state.zip'.format(thisyear))
os.remove(yeardir+'tl_{0:.0f}_us_county.zip'.format(thisyear))

print('Reading state names...')
statefile = yeardir+'tl_{0:.0f}_us_state.shp'.format(thisyear)
state_gdf = geopandas.read_file(statefile)
state_gdf = state_gdf.set_index('STATEFP')
state_gdf = state_gdf.sort_index()

e = time.time()
g += e-s
print('Done in {0:,.0f} minutes {1:,.0f} seconds!'.format(np.floor((e-s)/60), (e-s)%60))


getting state shapefiles...
getting county shapefiles...
Extracting state shapes zipfile...
Extracting county shapes zipfile...
Reading state names...
Done in 0 minutes 6 seconds!


# Get tract shapefiles

In [ ]:
s = time.time()
os.chdir(tractdir)

print('Downloading tract shapefiles...')
#totalfilesize = 0
for ix in sorted(state_gdf.index.tolist()):
    si = time.time()
    thisfilename = 'tl_{0:.0f}_{1:}_tract.zip'.format(thisyear, ix)
    thisurl = 'https://www2.census.gov/geo/tiger/TIGER{0:}/TRACT/{1:}'.format(thisyear, thisfilename)
    print('\tGetting {0:}...'.format(thisfilename))
    
    r = c.request('GET', thisurl, preload_content=False)

    with open(thisfilename, 'wb') as out:
        while True:
            data = r.read()
            if not data:
                break
            out.write(data)
            
    thisfilesize = os.stat(thisfilename).st_size
    ei = time.time()
    #print('\t{0:,.1f} MB in {1:,.0f} minutes {2:,.0f} seconds ({3:.1f} MB/s)...'.format(thisfilesize / (1024 * 1024), np.floor((ei-si)/60), (ei-si)%60, thisfilesize/(ei-si)))
    #totalfilesize += thisfilesize

r.release_conn()

print('\n')
print('Unzipping all files...')

for thefilename in sorted([x for x in os.listdir() if '.zip' in x]):
    print('\tExtracting zipfile {0:}...'.format(thefilename))
    thezipfile = zipfile.ZipFile(thefilename)
    thezipfile.extractall()
    thezipfile.close()
    os.remove(thefilename)

print('Calculating size of all files...')
totalfilesize = 0
for path, dirs, files in os.walk(os.getcwd()):
    for f in files:
        fp = os.path.join(path, f)
        totalfilesize += os.path.getsize(fp)

e = time.time()
g += e-s
print('\nDownloaded and unzipped {0:,.0f} MB total in {1:,.0f} minutes {2:,.0f} seconds!'.format(totalfilesize/(1024 * 1024), np.floor((e-s)/60), (e-s)%60))

# *_06_tract* *_11_tract* *_24_tract* *_26_tract* *_29_tract* *_34_tract* *39_tract* *_42_tract* *_51_tract*

# Block Groups

In [ ]:
# s = time.time()
# os.chdir(bgdir)
# #os.listdir()

# print('Downloading block group shapefiles...')
# #totalfilesize = 0
# for ix in sorted(state_gdf.index.tolist()):
# #    si = time.time()
#     thisfilename = 'tl_{0:.0f}_{1:}_bg.zip'.format(thisyear, ix)
#     thisurl = 'https://www2.census.gov/geo/tiger/TIGER{0:}/BG/{1:}'.format(thisyear, thisfilename)
#     print('\tGetting {0:}...'.format(thisfilename))
    
#     r = c.request('GET', thisurl, preload_content=False)

#     with open(thisfilename, 'wb') as out:
#         while True:
#             data = r.read()
#             if not data:
#                 break
#             out.write(data)
            
# #    thisfilesize = os.stat(thisfilename).st_size
# #    ei = time.time()
# #    print('\t{0:,.1f} MB in {1:,.0f} minutes {2:,.0f} seconds ({3:.1f} MB/s)...'.format(thisfilesize / (1024 * 1024), np.floor((ei-si)/60), (ei-si)%60, thisfilesize/(ei-si)))
# #    totalfilesize += thisfilesize

# r.release_conn()

# print('\n')
# print('Unzipping all files...')

# for thefilename in sorted([x for x in os.listdir() if '.zip' in x]):
#     print('\tExtracting zipfile {0:}...'.format(thefilename))
#     thezipfile = zipfile.ZipFile(thefilename)
#     thezipfile.extractall()
#     thezipfile.close()
#     os.remove(thefilename)

# print('Calculating size of all files...')
# totalfilesize = 0
# for path, dirs, files in os.walk(os.getcwd()):
#     for f in files:
#         fp = os.path.join(path, f)
#         totalfilesize += os.path.getsize(fp)

# e = time.time()
# g += e-s
# print('\nDownloaded and unzipped {0:,.0f} MB total in {1:,.0f} minutes {2:,.0f} seconds!'.format(totalfilesize/(1024 * 1024), np.floor((e-s)/60), (e-s)%60))


# Places

In [ ]:
s = time.time()
os.chdir(placedir)

print('Downloading place shapefiles...')
# totalfilesize = 0
for ix in sorted(state_gdf.index.tolist()):
#     si = time.time()
    thisfilename = 'tl_{0:.0f}_{1:}_place.zip'.format(thisyear, ix)
    thisurl = 'https://www2.census.gov/geo/tiger/TIGER{0:}/PLACE/{1:}'.format(thisyear, thisfilename)
    print('\tGetting {0:}...'.format(thisfilename))
    
    r = c.request('GET', thisurl, preload_content=False)

    with open(thisfilename, 'wb') as out:
        while True:
            data = r.read()
            if not data:
                break
            out.write(data)
            
#     thisfilesize = (os.stat(thisfilename).st_size)/(1024*1024)
#     ei = time.time()
#     print('\t\t{0:,.1f} MB in {1:,.0f} minutes {2:,.0f} seconds ({3:.1f} MB/s)...'.format(thisfilesize, np.floor((ei-si)/60), (ei-si)%60, thisfilesize / (ei-si)))
#     totalfilesize += thisfilesize

r.release_conn()

print('\n')
print('Unzipping all files...')

for thefilename in sorted([x for x in os.listdir() if '.zip' in x]):
#    print('Extracting zipfile {0:}...'.format(thefilename))
    thezipfile = zipfile.ZipFile(thefilename)
    thezipfile.extractall()
    thezipfile.close()
    os.remove(thefilename)

print('Calculating size of all files...')
totalfilesize = 0
for path, dirs, files in os.walk(os.getcwd()):
    for f in files:
        fp = os.path.join(path, f)
        totalfilesize += os.path.getsize(fp)

e = time.time()
g += e-s
print('\nDownloaded and unzipped {0:,.0f} MB total in {1:,.0f} minutes {2:,.0f} seconds!'.format(totalfilesize/(1024 * 1024), np.floor((e-s)/60), (e-s)%60))
# *_06_place* *_11_place* *_24_place* *_26_place* *_29_place* *_34_place* *39_place* *_42_place* *_51_place*

# Get urban area data

Known as community-based statistical area (CBSA) data in 2010

In [ ]:
# s = time.time()
# os.chdir(uacdir)
# os.getcwd()


# print('Downloading urban area shapefiles...')
# print('\n')
# if (thisyear <= 2010):
#     print('NEED CODE FOR 2010!')
# else:
#     possible_filenames_list = ['tl_{0:.0f}_us_cbsa00.zip'.format(thisyear), 'tl_{0:.0f}_us_uac10.zip'.format(thisyear), 'tl_{0:.0f}_us_uac20.zip'.format(thisyear)]

# for thisfilename in possible_filenames_list:
#     thisurl = 'https://www2.census.gov/geo/tiger/TIGER{0:}/UAC/{1:}'.format(thisyear, thisfilename)
    
#     print('Checking for existence of {0:}...'.format(thisfilename))
#     r = c.request('GET', thisurl, preload_content=False)
    
#     if (r.status == 404):
#         print('\tFile does not exist!')
#     else:
#         print('\tFile exists - downloading!')
#         with open(thisfilename, 'wb') as out:
#             while True:
#                 data = r.read()
#                 if not data:
#                     break
#                 out.write(data)

# r.release_conn()

# print('\n')
# print('Unzipping all files...')

# for thefilename in sorted([x for x in os.listdir() if '.zip' in x]):
# #    print('Extracting zipfile {0:}...'.format(thefilename))
#     thezipfile = zipfile.ZipFile(thefilename)
#     thezipfile.extractall()
#     thezipfile.close()
#     os.remove(thefilename)

# print('Calculating size of all files...')
# totalfilesize = 0
# for path, dirs, files in os.walk(os.getcwd()):
#     for f in files:
#         fp = os.path.join(path, f)
#         totalfilesize += os.path.getsize(fp)

# e = time.time()
# g += e-s
# print('\nDownloaded and unzipped {0:,.0f} MB total in {1:,.0f} minutes {2:,.0f} seconds!'.format(totalfilesize/(1024 * 1024), np.floor((e-s)/60), (e-s)%60))


## Congressional districts

In [ ]:
# s = time.time()
# os.chdir(cddir)

# print('Downloading congressional district shapefiles...')
# print('\n')
# congress_numbers = list(range(110,120,1))
# possible_filenames_list = ['tl_{0:.0f}_us_cd{1:.0f}.zip'.format(thisyear, x) for x in congress_numbers]
# possible_filenames_list

# for thisfilename in possible_filenames_list:
#     thisurl = 'https://www2.census.gov/geo/tiger/TIGER{0:}/CD/{1:}'.format(thisyear, thisfilename)
#     print(thisurl)
#     print('Checking for existence of {0:}...'.format(thisfilename))
#     r = c.request('GET', thisurl, preload_content=False)
    
#     if (r.status == 404):
#         print('\tFile does not exist!')
#     else:
#         print('\tFile exists - downloading!')
#         with open(thisfilename, 'wb') as out:
#             while True:
#                 data = r.read()
#                 if not data:
#                     break
#                 out.write(data)

# r.release_conn()

# print('\n')
# print('Unzipping all files...')

# for thefilename in sorted([x for x in os.listdir() if '.zip' in x]):
# #    print('Extracting zipfile {0:}...'.format(thefilename))
#     thezipfile = zipfile.ZipFile(thefilename)
#     thezipfile.extractall()
#     thezipfile.close()
#     os.remove(thefilename)

# print('Calculating size of all files...')
# totalfilesize = 0
# for path, dirs, files in os.walk(os.getcwd()):
#     for f in files:
#         fp = os.path.join(path, f)
#         totalfilesize += os.path.getsize(fp)

# e = time.time()
# g += e-s
# print('\nDownloaded and unzipped {0:,.0f} MB total in {1:,.0f} minutes {2:,.0f} seconds!'.format(totalfilesize/(1024 * 1024), np.floor((e-s)/60), (e-s)%60))


## Zip codes (ZCTA5)

In [ ]:
# s = time.time()
# os.chdir(zcta5dir)

# print('Downloading ZIP code (ZCTA5) shapefiles...')
# print('\n')

# if (thisyear >= 2020):
#     thisfilename = 'tl_{0:.0f}_us_zcta520.zip'.format(thisyear)
#     thisurl = 'https://www2.census.gov/geo/tiger/TIGER{0:.0f}/ZCTA520/{1:}'.format(thisyear, thisfilename)
# else:
#     thisfilename = 'tl_{0:.0f}_us_zcta510.zip'.format(thisyear)
#     thisurl = 'https://www2.census.gov/geo/tiger/TIGER{0:.0f}/ZCTA5/{1:}'.format(thisyear, thisfilename)
    
# print('Getting {0:}...'.format(thisfilename))

# r = c.request('GET', thisurl, preload_content=False)
# with open(thisfilename, 'wb') as out:
#     while True:
#         data = r.read()
#         if not data:
#             break
#         out.write(data)
# r.release_conn()

# print('\n')
# print('Unzipping all files...')
# for thisfilename in sorted([x for x in os.listdir() if '.zip' in x]):
#     print(thisfilename)
#     print('Extracting zipfile {0:}...'.format(thisfilename))
#     thezipfile = zipfile.ZipFile(thisfilename)
#     thezipfile.extractall()
#     thezipfile.close()
#     os.remove(thisfilename)

# print('Calculating size of all files...')
# totalfilesize = 0
# for path, dirs, files in os.walk(os.getcwd()):
#     for f in files:
#         fp = os.path.join(path, f)
#         totalfilesize += os.path.getsize(fp)

# e = time.time()
# g += e-s
# print('\nDownloaded and unzipped {0:,.0f} MB total in {1:,.0f} minutes {2:,.0f} seconds!'.format(totalfilesize/(1024 * 1024), np.floor((e-s)/60), (e-s)%60))


# Get water area shapefiles: AREAWATER

In [56]:
s = time.time()

# do_these_states = ['06', '11', '24', '26', '29', '34', '39', '42'
# #      '01', '02', '04', '05',
# #      '06',
# #     '08', '09', '10', '11', '12' , 
# #      '13', '15', '16', '17', '18', 
# #      '19', '20', '21', '22', '23', '24',
# #      '25', '26', '27', '28', 
# #      '29', '30', '31', '32', '33', 
# #      '34', '35', '36', '37', '38', 
# #      '39', '40', '41', '42', '44', 
# #      '45', '46', '47', 
# #     '48', 
# #    '49', '50', '51', '53', 
# #      '54', '55', '56', '60', '66', '69', '72', '78'
#     ]


os.chdir(areawaterdir)

print('Finding all downloadable files...')
listurl = 'https://www2.census.gov/geo/tiger/TIGER{0:.0f}/AREAWATER/'.format(thisyear)

resp = c.request("GET", listurl)
soup = bs(resp.data, 'html.parser')
thetable = soup.find('table')
all_file_names = []
for i in range(0, len(thetable)):
    if (len(thetable.contents[i]) > 1):
        tdlist = thetable.contents[i].findAll('td')
        if (len(tdlist) > 1):
            all_file_names.append(tdlist[1].get_text())

print('Downloading area water shapefiles...')
print('\n')
for ix in do_these_states:
    print('Getting areawater files for {0:} (STUSPS = {1:})...'.format(state_gdf.loc[ix]['NAME'], ix))
    #print('\tCreating directory...')
    statedirname = areawaterdir+state_gdf.loc[ix]['STUSPS'].lower()
    if (os.path.exists(statedirname)):
        os.chdir(statedirname)
        for x in os.listdir():
            os.remove(x)
        os.chdir(areawaterdir)
        os.rmdir(statedirname)
    os.makedirs(statedirname)
    os.chdir(statedirname)
    #print('\tGetting files...')
    for thisfilename in [x for x in all_file_names if 'tl_{0:.0f}_{1:}'.format(thisyear, ix) in x]:
        thisurl = 'https://www2.census.gov/geo/tiger/TIGER{0:}/AREAWATER/{1:}'.format(thisyear, thisfilename)
        #print('\t\t{0:}'.format(thisfilename))
        r = c.request('GET', thisurl, preload_content=False)
        with open(thisfilename, 'wb') as out:
            while True:
                data = r.read()
                if not data:
                    break
                out.write(data)
    os.chdir(areawaterdir)
r.release_conn()

print('Calculating size of zip files...')
nfiles = 0
totalfilesize = 0
for path, dirs, files in os.walk(areawaterdir):
    for f in files:
        fp = os.path.join(path, f)
        nfiles += 1
        totalfilesize += os.path.getsize(fp)

e = time.time()
g += e-s
print('\nDownloaded {0:,.0f} files ({1:,.0f} MB total) in {2:,.0f} minutes {3:,.0f} seconds!'.format(nfiles, totalfilesize/(1024 * 1024), np.floor((e-s)/60), (e-s)%60))

for ix in do_these_states:
    print('\nUnzipping areawater files for {0:} (STUSPS = {1:})...'.format(state_gdf.loc[ix]['NAME'], ix))
    
    dirname = '{0:}{1:}'.format(areawaterdir, state_gdf.loc[ix]['STUSPS'].lower())
    os.chdir(dirname)
    zipfiles = [x for x in os.listdir(dirname) if x[-4:] == '.zip']
    for i in range(0, len(zipfiles)):
        if (np.mod(i,10) == 0):
            print('\tUnzipping {0:,.0f} of {1:,.0f}...'.format(i, len(zipfiles)))
        thezipfile = zipfile.ZipFile(zipfiles[i])
        thezipfile.extractall()
        thezipfile.close()
        os.remove(zipfiles[i])
    os.chdir(areawaterdir)
print('\n')
print('Calculating size of unzipped files...')
nfiles = 0
totalfilesize = 0
for path, dirs, files in os.walk(areawaterdir):
    for f in files:
        fp = os.path.join(path, f)
        nfiles += 1
        totalfilesize += os.path.getsize(fp)

e = time.time()
g += e-s
print('\nUnzipped {0:,.0f} files with {1:,.0f} MB total in {2:,.0f} minutes {3:,.0f} seconds!'.format(nfiles, totalfilesize/(1024 * 1024), np.floor((e-s)/60), (e-s)%60))
# 01-24: Unzipped 8,533 files with 910 MB total in 4 minutes 0 seconds!
# 25-47: Unzipped 17,675 files with 1,743 MB total in 4 minutes 14 seconds!
# 48: Unzipped 19,453 files with 2,020 MB total in 1 minutes 5 seconds!
# 49-78: Unzipped 22,645 files with 2,285 MB total in 1 minutes 28 seconds!



Finding all downloadable files...


Getting areawater files for California (STUSPS = 06)...
Getting areawater files for District of Columbia (STUSPS = 11)...
Getting areawater files for Maryland (STUSPS = 24)...
Getting areawater files for Michigan (STUSPS = 26)...
Getting areawater files for Missouri (STUSPS = 29)...
Getting areawater files for New Jersey (STUSPS = 34)...
Getting areawater files for Ohio (STUSPS = 39)...
Getting areawater files for Pennsylvania (STUSPS = 42)...
Getting areawater files for Virginia (STUSPS = 51)...
Calculating size of zip files...

Downloaded 591 files (202 MB total) in 0 minutes 55 seconds!

Unzipping areawater files for California (STUSPS = 06)...
	Unzipping 0 of 58...
	Unzipping 10 of 58...
	Unzipping 20 of 58...
	Unzipping 30 of 58...
	Unzipping 40 of 58...
	Unzipping 50 of 58...

Unzipping areawater files for District of Columbia (STUSPS = 11)...
	Unzipping 0 of 1...

Unzipping areawater files for Maryland (STUSPS = 24)...
	Unzipping 0 of 24...
	U

# ROADS

In [57]:
s = time.time()
os.chdir(roaddir)


# do_these_states = ['06', '11', '24', '26', '29', '34', '39', '42'    
# #      '01', '02', '04', '05',
# #      '06',
# #     '08', '09', '10', '11', '12' , 
# #      '13', '15', '16', '17', '18', 
# #      '19', '20', '21', '22', '23', '24',
# #      '25', '26', '27', '28', 
# #      '29', '30', '31', '32', '33', 
# #      '34', '35', '36', '37', '38', 
# #      '39', '40', '41', '42', '44', 
# #      '45', '46', '47', 
# #    '48', 
# #    '49', '50', '51', '53', 
# #      '54', '55', '56', '60', '66', '69', '72', '78'
#     ]




print('Finding all downloadable files...')
listurl = 'https://www2.census.gov/geo/tiger/TIGER{0:.0f}/ROADS/'.format(thisyear)

resp = c.request("GET", listurl)
soup = bs(resp.data, 'html.parser')
thetable = soup.find('table')
all_file_names = []
for i in range(0, len(thetable)):
    if (len(thetable.contents[i]) > 1):
        tdlist = thetable.contents[i].findAll('td')
        if (len(tdlist) > 1):
            all_file_names.append(tdlist[1].get_text())

print('Downloading road shapefiles...')
print('\n')

for ix in do_these_states:
    print('Getting road files for {0:} (STUSPS = {1:})...'.format(state_gdf.loc[ix]['NAME'], ix))
    statedirname = roaddir+state_gdf.loc[ix]['STUSPS'].lower()
    if (os.path.exists(statedirname)):
        os.chdir(statedirname)
        for x in os.listdir():
            os.remove(x)
        os.chdir(roaddir)
        os.rmdir(statedirname)
    os.makedirs(statedirname)
    os.chdir(statedirname)
    #print('\tGetting files...')
    for thisfilename in [x for x in all_file_names if 'tl_{0:.0f}_{1:}'.format(thisyear, ix) in x]:
        thisurl = 'https://www2.census.gov/geo/tiger/TIGER{0:}/ROADS/{1:}'.format(thisyear, thisfilename)
        #print(thisurl)
#        print('\t\t{0:}'.format(thisfilename))
        r = c.request('GET', thisurl, preload_content=False)
        with open(thisfilename, 'wb') as out:
            while True:
                data = r.read()
                if not data:
                    break
                out.write(data)
    os.chdir(roaddir)
r.release_conn()
print('\n')
print('Unzipping...')
nfiles = 0
totalfilesize = 0
for ix in do_these_states:
    print('Unzipping road files for {0:} (STUSPS = {1:})...'.format(state_gdf.loc[ix]['NAME'], ix))
    
    dirname = '{0:}{1:}'.format(roaddir, state_gdf.loc[ix]['STUSPS'].lower())
    os.chdir(dirname)
    zipfiles = [x for x in os.listdir(dirname) if x[-4:] == '.zip']
    for i in range(0, len(zipfiles)):
        if (np.mod(i,10) == 0):
            print('\tUnzipping {0:,.0f} of {1:,.0f}...'.format(i, len(zipfiles)))
        thezipfile = zipfile.ZipFile(zipfiles[i])
        thezipfile.extractall()
        thezipfile.close()
        os.remove(zipfiles[i])
    os.chdir(roaddir)

e = time.time()
g += e-s
#print('\nUnzipped {0:,.0f} files with {1:,.0f} MB total in {2:,.0f} minutes {3:,.0f} seconds!'.format(nfiles, totalfilesize/(1024 * 1024), np.floor((e-s)/60), (e-s)%60))
print('\nDone in {0:,.0f} minutes {1:,.0f} seconds!'.format(np.floor((e-s)/60), (e-s)%60))


Finding all downloadable files...


Getting road files for California (STUSPS = 06)...
Getting road files for District of Columbia (STUSPS = 11)...
Getting road files for Maryland (STUSPS = 24)...
Getting road files for Michigan (STUSPS = 26)...
Getting road files for Missouri (STUSPS = 29)...
Getting road files for New Jersey (STUSPS = 34)...
Getting road files for Ohio (STUSPS = 39)...
Getting road files for Pennsylvania (STUSPS = 42)...
Getting road files for Virginia (STUSPS = 51)...


Unzipping...
Unzipping road files for California (STUSPS = 06)...
	Unzipping 0 of 58...
	Unzipping 10 of 58...
	Unzipping 20 of 58...
	Unzipping 30 of 58...
	Unzipping 40 of 58...
	Unzipping 50 of 58...
Unzipping road files for District of Columbia (STUSPS = 11)...
	Unzipping 0 of 1...
Unzipping road files for Maryland (STUSPS = 24)...
	Unzipping 0 of 24...
	Unzipping 10 of 24...
	Unzipping 20 of 24...
Unzipping road files for Michigan (STUSPS = 26)...
	Unzipping 0 of 83...
	Unzipping 10 of 83...
	Un

# PUMA

In [ ]:
# os.chdir(pumadir)

# if (thisyear == 2011):
#     print('No PUMA shapefiles for 2011!')
# else:
#     print('FTP-ing...')
#     ftp = FTP('ftp2.census.gov')
#     ftp.login()
#     print(ftp.getwelcome())
#     if (thisyear == 2010):
#         ftp.cwd('geo/tiger/TIGER{0:.0f}/PUMA5/{0:.0f}'.format(thisyear))
#         print(ftp.nlst())
#     else:
#         ftp.cwd('geo/tiger/TIGER{0:.0f}/PUMA/'.format(thisyear))
#     for i in range(0,100):    
#         statefiles = [x for x in ftp.nlst() if 'tl_{0:.0f}_{1:02d}'.format(thisyear, i) in x]
#         if len(statefiles) > 0:
#             print('Getting PUMA shapefiles for state {0:02d}...'.format(i))
#             for thisfile in statefiles:
#                 with open(thisfile, 'wb') as f:
#                     ftp.retrbinary('RETR {0:}'.format(thisfile), f.write)
#     ftp.close()


# os.chdir(pumadir)
# allzipfiles = sorted([x for x in os.listdir() if '.zip' in x])

# for thisfile in allzipfiles:
#     thezipfile = zipfile.ZipFile(thisfile)
#     thezipfile.extractall()
#     thezipfile.close()

# for thiszip in [x for x in os.listdir() if '.zip' in x]:
#     os.remove(thiszip)
# print('Done!')
# # print(os.getcwd())
# # os.listdir()
# print('Done!')